<a href="https://colab.research.google.com/github/jocelynbaduria/cmpe-297_SOTA/blob/main/Avalanche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1. Follow the same tutorial but use Omniglot Dataset instead of MNIST. Got error with input shape - RuntimeError: shape '[32, 784]' is invalid for input of size 352800.

Install Avalanche

In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-_r65zp9m
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-_r65zp9m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 121 kB 31.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 41.3 MB/s 
     |████████████████████████████████| 1.7 MB 29.6 MB/s 
     |████████████████████████████████| 139 kB 60.5 MB/s 
     |████████████████████████████████| 180 kB 52.1 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=426447 sha256=758b0213d03562b96aa95748287beeffa04562c445a82b42a461cf6a

In [2]:
!pip install torchvision

In [3]:
import torchvision

dataset = torchvision.datasets.Omniglot(
    root="./data", download=True, transform=torchvision.transforms.ToTensor()
)
image, label = dataset[0]
print(type(image))  # torch.Tensor
print(type(label))  # int

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting ./data/omniglot-py/images_background.zip to ./data/omniglot-py
<class 'torch.Tensor'>
<class 'int'>


Import Omniglot Dataset

In [4]:
from avalanche.benchmarks.datasets import Omniglot

In [5]:
from avalanche.benchmarks.classic import SplitOmniglot

In [6]:
# creating the benchmark (scenario object)
split_omniglot = SplitOmniglot(
    n_experiences=4,
    seed=1234,
)
# recovering the train and test streams
train_stream = split_omniglot.train_stream
test_stream = split_omniglot.test_stream
# iterating over the train stream
for experience in train_stream:
    print("Start of task ", experience.task_label)
    print('Classes in this task:', experience.classes_in_this_experience)
    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    # ...as well as the task_label
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')
    # we can recover the corresponding test experience in the test stream
    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')

  0%|          | 0/9464212 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/omniglot/omniglot-py/omniglot-py/images_background.zip to /root/.avalanche/data/omniglot/omniglot-py/omniglot-py


  0%|          | 0/6462886 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/omniglot/omniglot-py/omniglot-py/images_evaluation.zip to /root/.avalanche/data/omniglot/omniglot-py/omniglot-py
Start of task  0
Classes in this task: [5, 517, 519, 8, 521, 523, 528, 529, 19, 23, 538, 29, 544, 35, 550, 40, 44, 557, 560, 49, 50, 51, 562, 565, 567, 568, 569, 58, 59, 571, 63, 64, 576, 68, 70, 584, 587, 588, 81, 82, 85, 86, 90, 91, 603, 605, 606, 607, 96, 97, 608, 609, 613, 615, 616, 106, 623, 626, 115, 116, 117, 118, 122, 637, 126, 127, 128, 129, 640, 643, 644, 136, 137, 140, 653, 142, 143, 145, 658, 148, 666, 157, 158, 669, 672, 161, 164, 677, 167, 681, 682, 171, 173, 175, 176, 687, 178, 690, 180, 181, 182, 693, 694, 697, 704, 195, 198, 710, 201, 718, 720, 724, 725, 728, 730, 731, 220, 221, 733, 734, 737, 227, 232, 744, 238, 242, 755, 759, 251, 763, 257, 772, 262, 776, 266, 267, 786, 787, 276, 277, 789, 790, 794, 795, 284, 285, 796, 797, 799, 801, 804, 293, 805, 807, 301, 813, 815, 817, 306, 307, 309, 822, 823, 315, 831, 832, 323, 325, 3

Run the MLP Model

In [7]:
################################################################################
# Copyright (c) 2021 ContinualAI.                                              #
# Copyrights licensed under the MIT License.                                   #
# See the accompanying LICENSE file for terms.                                 #
#                                                                              #
# Date: 1-05-2020                                                              #
# Author(s): Vincenzo Lomonaco, Antonio Carta                                  #
# E-mail: contact@continualai.org                                              #
# Website: avalanche.continualai.org                                           #
################################################################################

import torch.nn as nn

from avalanche.models.dynamic_modules import MultiTaskModule, \
    MultiHeadClassifier
from avalanche.models.base_model import BaseModel


class SimpleMLP(nn.Module, BaseModel):
    def __init__(self, num_classes=10, batch_size=32, input_size=28 * 28,
                 hidden_size=512, hidden_layers=1, drop_rate=0.5):
        super().__init__()

        layers = nn.Sequential(*(nn.Linear(input_size, hidden_size),
                                 nn.ReLU(inplace=True),
                                 nn.Dropout(p=drop_rate)))
        for layer_idx in range(hidden_layers - 1):
            layers.add_module(
                f"fc{layer_idx + 1}", nn.Sequential(
                    *(nn.Linear(hidden_size, hidden_size),
                      nn.ReLU(inplace=True),
                      nn.Dropout())))

        self.features = nn.Sequential(*layers)
        self.classifier = nn.Linear(hidden_size, num_classes)
        self._input_size = input_size
        self._batch_size = batch_size

    def forward(self, x):
        x = x.contiguous()
        # x = x.view(-1, 1 * 28 * 28)
        x = x.view(x.size(0), self._input_size)
        x = x.view(x.size(0), self._batch_size, self._input_size)
        print(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

    def get_features(self, x):
        x = x.contiguous()
        x = x.view(x.size(0), self._input_size)
        x = self.features(x)
        return x


class MTSimpleMLP(MultiTaskModule):
    def __init__(self, input_size=28 * 28, hidden_size=512):
        """
            Multi-task MLP with multi-head classifier.
        """
        super().__init__()

        self.features = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Dropout(),
        )
        self.classifier = MultiHeadClassifier(hidden_size)
        self._input_size = input_size

    def forward(self, x, task_labels):
        x = x.contiguous()
        x = x.view(x.size(0), self._input_size)
        x = self.features(x)
        x = self.classifier(x, task_labels)
        return x


__all__ = [
    'SimpleMLP',
    'MTSimpleMLP'
]

Train the Omniglot Dataset with SimpleMLP model

In [8]:
from avalanche.benchmarks.datasets import Omniglot
from avalanche.benchmarks.classic import SplitOmniglot
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

from torch.optim import SGD
from torch.nn import CrossEntropyLoss


benchmark = SplitOmniglot(n_experiences=4, seed=1)
# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns 
# them to the strategy it is attached to.


# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)


# Than we can extract the parallel train and test streams
train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, optimizer, criterion,
    train_mb_size=32, train_epochs=2, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=2)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=4))

Files already downloaded and verified
Files already downloaded and verified
Starting experiment...
Start of experience:  0
Current Classes:  [2, 8, 520, 10, 529, 531, 534, 28, 541, 542, 33, 35, 548, 550, 40, 554, 556, 47, 559, 561, 564, 567, 568, 58, 571, 573, 62, 574, 64, 580, 70, 72, 74, 589, 78, 590, 592, 593, 594, 595, 598, 87, 600, 90, 93, 607, 97, 100, 102, 614, 617, 107, 623, 626, 115, 628, 122, 123, 636, 126, 638, 129, 647, 139, 651, 653, 143, 144, 145, 146, 655, 148, 656, 657, 659, 152, 664, 668, 157, 161, 163, 676, 166, 679, 168, 681, 171, 685, 174, 686, 693, 696, 698, 187, 699, 191, 705, 706, 195, 196, 707, 708, 199, 200, 710, 202, 716, 721, 211, 212, 213, 728, 729, 218, 733, 223, 738, 739, 228, 229, 233, 241, 756, 761, 762, 763, 252, 253, 764, 255, 766, 770, 774, 775, 264, 778, 267, 269, 271, 784, 786, 789, 791, 283, 284, 796, 287, 289, 801, 804, 805, 295, 809, 812, 813, 308, 309, 310, 821, 312, 827, 829, 318, 833, 322, 835, 325, 837, 327, 840, 841, 330, 331, 847, 337, 849,

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


0it [00:00, ?it/s]

RuntimeError: ignored

###2. Use MNIST Tutorial Dataset

Import MNIST dataset

In [9]:
from avalanche.benchmarks.datasets import MNIST

In [11]:
from avalanche.benchmarks.classic import SplitMNIST

Train the MNIST Dataset with SimpleMLP model

In [13]:
# from avalanche.benchmarks.datasets import MNIST
# from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics,\
    loss_metrics, timing_metrics, cpu_usage_metrics, StreamConfusionMatrix,\
    disk_usage_metrics, gpu_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

from torch.optim import SGD
from torch.nn import CrossEntropyLoss


benchmark = SplitMNIST(n_experiences=5, seed=1)
# MODEL CREATION
model = SimpleMLP(num_classes=benchmark.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns 
# them to the strategy it is attached to.


# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=benchmark.n_classes, save_image=False),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)


# Than we can extract the parallel train and test streams
train_stream = benchmark.train_stream
test_stream = benchmark.test_stream

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, optimizer, criterion,
    train_mb_size=32, train_epochs=2, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience, num_workers=2)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # eval also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream, num_workers=4))

Starting experiment...
Start of experience:  0
Current Classes:  [5, 6]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


100%|██████████| 355/355 [02:37<00:00,  2.26it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 69557.2930
	DiskUsage_MB/train_phase/train_stream/Task000 = 69557.2930
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1867
	Loss_MB/train_phase/train_stream/Task000 = 0.0229
	Time_Epoch/train_phase/train_stream/Task000 = 156.6928
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9464
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████| 355/355 [02:38<00:00,  2.25it/s]
Epoch 1 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 69642.8877
	DiskUsage_MB/train_phase/train_stream/Task000 = 69642.8877
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0731
	Loss_MB/train_phase/train_stream/Task000 = 0.0519
	Time_Epoch/train_phase/train_stream/Task000 = 157.5533
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9757
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
Training completed
Computing accuracy on t

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 19/19 [00:08<00:00,  2.19it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 96.8896
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 69643.4150
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0610
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9795
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 22/22 [00:10<00:00,  2.16it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 97.0633
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 69643.7705
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 8.2751
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0000
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 20/20 [00:09<00:00,  2.15it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002 = 96.8269
	DiskUsage